In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
import random
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
from mpl_toolkits.mplot3d import Axes3D
writer = SummaryWriter()

In [2]:
DATA_PATH = 'C:/Users/user/Desktop/THESIS YLIKO/batch9.txt'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

c:\users\user\pymol\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [3]:
def load_data(path):
    # read in from csv
    expressions = pd.read_csv(DATA_PATH, sep=',')
    subtypes = expressions['Batch']
    subtypes
    expressions = expressions.drop(columns=['x','Batch', 'Cell','Group']).astype('float32')

    index = list(subtypes.index[subtypes == 'Batch1']) + \
    list(subtypes.index[subtypes == 'Batch2']) + \
    list(subtypes.index[subtypes == 'Batch3'])
    
    #list(subtypes.index[subtypes == 'Batch4']) + \
    
    #random.shuffle(index)
    #expressions = expressions.iloc[index, :].astype('float32')
    #subtypes = subtypes.iloc[index]
    #y=y.iloc[index]
    standardizer = preprocessing.StandardScaler()
    expressions = standardizer.fit_transform(expressions) 
    
    return expressions, standardizer, subtypes#, y
    
def numpyToTensor(expressions):
    x_train = torch.from_numpy(expressions).to(device)
    return x_train

In [4]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path):
        self.expressions, self.standardizer,self.subtypes= load_data(DATA_PATH)
        self.expressions = numpyToTensor(self.expressions)
        self.len=self.expressions.shape[0]
        
    def __getitem__(self, item):
        return self.expressions[item],self.expressions[item]

    def __len__(self):
        return self.len

In [5]:
data_set = DataBuilder(DATA_PATH)
trainloader = DataLoader(dataset=data_set, batch_size=30)

In [6]:
class Autoencoder(nn.Module):
    def __init__(self, D_in, H, H2, latent_dim=20):
        
        # Encoder
        super(Autoencoder, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2 = nn.Linear(H, H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3 = nn.Linear(H2, H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        

        #         # Latent vectors mu and sigma
        #X
        self.fc1 = nn.Linear(H2, latent_dim+latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim+latent_dim)
        self.fc21 = nn.Linear(latent_dim+latent_dim, latent_dim+latent_dim)
        self.fc22 = nn.Linear(latent_dim+latent_dim, latent_dim+latent_dim)
        
        #D
        self.fc11 = nn.Linear(H2, latent_dim)
        self.bn11 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc211 = nn.Linear(latent_dim, latent_dim)
        self.fc222 = nn.Linear(latent_dim, latent_dim)

        #         # Sampling vector
        #X
        self.fc3 = nn.Linear(latent_dim+latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        #D
        self.fc33 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn33 = nn.BatchNorm1d(latent_dim)
        self.fc44 = nn.Linear(latent_dim, H2)
        self.fc_bn44 = nn.BatchNorm1d(H2)

        #         # Decoder
        self.linear4 = nn.Linear(H2, H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5 = nn.Linear(H2, H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6 = nn.Linear(H, D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)

        self.relu = nn.ReLU()

    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2
    
    def encode_d(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc11 = F.relu(self.bn11(self.fc11(lin3)))

        r1 = self.fc211(fc11)
        r2 = self.fc222(fc11)

        return r1, r2

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, zx, zd):
        z = torch.cat((zx, zd))
        print("LATENT VARIABLE SIZE: ",z.size())
        '''
        fc3 = self.relu(self.fc_bn3(self.fc3(zx)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))
        '''
        fc33 = self.relu(self.fc_bn33(self.fc33(z)))
        fc44 = self.relu(self.fc_bn44(self.fc44(fc33)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc44)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))
    
    def decode_d(self, zd):
        fc33 = self.relu(self.fc_bn33(self.fc33(zd)))
        fc44 = self.relu(self.fc_bn44(self.fc44(fc33)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc44)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))

    def forward(self, x,d):
        mu, logvar = self.encode_d(x)
        z = self.reparameterize(mu, logvar)
        mu_d, logvar_d = self.encode_d(x)
        zd = self.reparameterize(mu_d, logvar_d)

        return self.decode(z,zd), mu, logvar,self.decode_d(zd), mu_d, logvar_d


In [7]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
        
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [8]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)

In [9]:
D_in = data_set.expressions.shape[1]
H = 30
H2 = 20
model = Autoencoder(D_in, H, H2).to(device)
model.apply(weights_init_uniform_rule)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

loss_mse = customLoss()

epochs = 4000
log_interval = 100
val_losses = []
train_losses = []

In [10]:
mu_output = []
mu_output_d = []
save_recon = []
save_data = []
def train(epoch):
    model.train()
    train_loss = 0
    trainloader
    for batch_idx, (x, d) in enumerate(trainloader):
        x = x.to(device)
        d = d.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar, recon_batch_d, mu_d, logvar_d = model(x, d) 
        print("RECONSTRUCTION SIZE: ",recon_batch.size(),"INPUT DATA SIZE: ",x.size())
        #x = torch.cat((x,d))
        
        mu_tensor = mu   
        mu_output.append(mu_tensor) 
        rec_tensor = recon_batch  
        save_recon.append(rec_tensor)
        
        mu_tensor_d = mu_d   
        mu_output_d.append(mu_tensor_d) 
        
        loss = loss_mse(recon_batch, x, mu, logvar)
        if(loss<0):
            print(loss)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    if epoch % 100 == 0:
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        writer.add_scalar("LOSS",train_loss / len(trainloader.dataset), epoch)
        train_losses.append(train_loss / len(trainloader.dataset))
        
        writer.add_histogram('linear1.weight', model.linear1.weight, epoch)
        writer.add_histogram('linear1.weight', model.linear2.weight, epoch)
        writer.add_histogram('linear3.weight', model.linear3.weight, epoch)
        
        writer.add_histogram('fc1.weight', model.fc1.weight, epoch)
        writer.add_histogram('fc21.weight', model.fc21.weight, epoch)
        writer.add_histogram('fc22.weight', model.fc22.weight, epoch)
        
        writer.add_histogram('fc3.weight', model.fc3.weight, epoch)
        writer.add_histogram('fc4.weight', model.fc4.weight, epoch)
        
        writer.add_histogram('linear4.weight', model.linear4.weight, epoch)
        writer.add_histogram('linear5.weight', model.linear5.weight, epoch)
        writer.add_histogram('linear6.weight', model.linear5.weight, epoch)

    if epoch == 3900:
        print("reconstruction",recon_batch.size())
        print("data",x.size())
        print("mu",mu.size())
for epoch in range(1, epochs + 1):
    train(epoch)
writer.flush()

LATENT VARIABLE SIZE:  torch.Size([60, 20])
RECONSTRUCTION SIZE:  torch.Size([60, 20]) INPUT DATA SIZE:  torch.Size([30, 20])


c:\users\user\pymol\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([30, 20])) that is different to the input size (torch.Size([60, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (60) must match the size of tensor b (30) at non-singleton dimension 0

In [ ]:
from mpl_toolkits import mplot3d

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt




x_pca, standardizer, data = load_data(DATA_PATH)
print(x_pca)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

fig,ax = plt.subplots()

xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
ax.scatter(xdata, ydata)

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
zdata = principalDf.iloc[:,2].values
ax.scatter(xdata, ydata,zdata)


In [ ]:

mu_pca = mu_output[3999].cpu().detach().numpy()
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(mu_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

fig,ax = plt.subplots()

xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
ax.scatter(xdata, ydata)

mu_pca = mu_output[3999].cpu().detach().numpy()

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(mu_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
zdata = principalDf.iloc[:,2].values
ax.scatter(xdata, ydata,zdata)


In [ ]:

res_pca = save_recon[3999].cpu().detach().numpy()
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(res_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

fig,ax = plt.subplots()

xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
ax.scatter(xdata, ydata)

res_pca = save_recon[3999].cpu().detach().numpy()

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(res_pca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


xdata = principalDf.iloc[:,0].values
ydata = principalDf.iloc[:,1].values
zdata = principalDf.iloc[:,2].values
ax.scatter(xdata, ydata,zdata)


In [ ]:
data
x_pca,mu_pca

In [ ]:
writer.add_embedding(x_pca,metadata=data,tag='data')
writer.add_embedding(mu_output[3999].cpu().detach().numpy(),metadata=data,tag='mu')
#writer.add_embedding(save_recon[3999].cpu().detach().numpy(),metadata=data,tag='recon')

writer.flush()
writer.close()

In [ ]:
plt.title("Reconstructed vs Input Data")
plt.plot(x_pca[0], color='blue', label='Input data')
plt.plot(save_recon[3999].cpu().detach().numpy()[0], color='red',  label='Reconstructed data') 
plt.legend();

In [ ]:
data= []
rec = []
gene = 9

plt.title("Correlation between Reconstructed and Input Data")


for i in range(30):
    data.append(x_pca[i][gene])
    rec.append(save_recon[3999].cpu().detach().numpy()[i][gene])

plt.plot(data,rec,'o',  label='Input data')